In [ ]:
from trl import DPOTrainer, DPOConfig
import torch
import random
import numpy as np

# from datasets.dataset_dict import DatasetDict
from datasets import Dataset
from evaluate import load
import numpy as np
# from datasets import load_dataset
import pandas as pd
import os
import logging
import warnings
import json
from huggingface_hub import login
import wandb
# import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    set_seed,
)

from torch.utils.data import IterableDataset, DataLoader
from trl import DPOTrainer, DPOConfig

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.FATAL)
torch.set_float32_matmul_precision('medium') 
logging.getLogger("lightning").setLevel(logging.CRITICAL)
logging.getLogger("lightning.fabric.utilities.rank_zero").setLevel(logging.CRITICAL)
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.CRITICAL)


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-05-28 21:53:21.847210: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-28 21:53:23.291867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-28 21:53:26.827766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
CONFIG = json.load(open('/home/jupyter/datasphere/project/tokens.json'))
tok = CONFIG["HF_TOK"]
wdb_tok = CONFIG['WANDB_API_KEY']

login(token=tok)
wandb.login(key=wdb_tok)

In [3]:
dev = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.dev.tsv'
train = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.train.tsv'
test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv'

rulec_train = pd.read_csv(train, delimiter='\t')
rulec_test = pd.read_csv(test, delimiter='\t')
rulec_dev = pd.read_csv(dev, delimiter='\t')

clang8 = pd.read_csv('/home/jupyter/datasphere/project/rugec/data/clang8_source_target_ru.spacy_tokenized.tsv', sep='\t')
gera_train = pd.read_csv('/home/jupyter/datasphere/project/rugec/data/GERA.train.tsv', sep='\t')
gera_test = pd.read_csv('/home/jupyter/datasphere/project/rugec/data/GERA.test.tsv', sep='\t')
gera_dev = pd.read_csv('/home/jupyter/datasphere/project/rugec/data/GERA.dev.tsv', sep='\t')
relco = pd.read_csv('/home/jupyter/datasphere/project/rugec/data/relco_filtered.tsv', sep='\t')

# train.corrupt_sent = train.corrupt_sent.map(lambda x: str(x))
# test.corrupt_sent = test.corrupt_sent.map(lambda x: str(x))
# dev.corrupt_sent = dev.corrupt_sent.map(lambda x: str(x))
# train.correct_sent = train.correct_sent.map(lambda x: str(x))
# test.correct_sent = test.correct_sent.map(lambda x: str(x))
# dev.correct_sent = dev.correct_sent.map(lambda x: str(x))

# train_all = pd.concat([rulec_train.sample(frac=10, replace=True), clang8, relco])
train_all = pd.concat([rulec_train, clang8, relco, gera_train, gera_dev])
train_all.corrupt_sent = train_all.corrupt_sent.map(lambda x: str(x))
train_all.correct_sent = train_all.correct_sent.map(lambda x: str(x))
train_all = train_all.sample(frac=1)
fine_tune = {'train':Dataset.from_dict({'corrupt_sent':train_all['corrupt_sent'],'correct_sent' : train_all['correct_sent']}),
     'test':Dataset.from_dict({'corrupt_sent':rulec_test['corrupt_sent'],'correct_sent' : rulec_test['correct_sent']}),
      'dev':Dataset.from_dict({'corrupt_sent' : rulec_dev['corrupt_sent'], 'correct_sent':rulec_dev['correct_sent']})
     }
fine_tune

{'train': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 59947
 }),
 'test': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 5000
 }),
 'dev': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 2500
 })}

In [4]:
def get_model_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    else:
        return torch.device("cpu")
    
device = get_model_device()
device

device(type='cuda')

In [5]:
name = 'mika5883/ft_rugec_A'
config = AutoConfig.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForSeq2SeqLM.from_pretrained(name).to(device)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
comet_metric = load('comet') 

Fetching 5 files: 100%|██████████| 5/5 [00:40<00:00,  8.16s/it]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../../tmp/xdg_cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.


In [ ]:
class OnTheFlyDpoDataset(IterableDataset):
    def __init__(self, prompts, golds, model, tokenizer, comet_metric, num_beams=6, num_beam_groups=3, batch_size=8):
        self.prompts = prompts
        self.golds = golds
        self.model = model
        self.tokenizer = tokenizer
        self.comet = comet_metric
        self.num_beams = num_beams
        self.num_beam_groups = num_beam_groups
        self.batch_size = batch_size
        self.ep = 0
        self.save_data = {}
    def __len__(self):
        return len(self.prompts)
    def __iter__(self):
        for i in range(0, len(self.prompts), self.batch_size):
            batch_prompts = self.prompts[i : i + self.batch_size]
            batch_golds   = self.golds[i : i + self.batch_size]

            inputs = self.tokenizer(
                batch_prompts,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=128,
            ).to(self.model.device)
            # outputs = self.model.generate(
            #     **inputs,
            #     num_beams=self.num_beams,
            #     num_return_sequences=self.num_beams,
            #     max_length=128,
            # )
            outputs = self.model.generate(
                **inputs,
                num_beams=self.num_beams,
                num_return_sequences=self.num_beams,
                num_beam_groups = self.num_beam_groups,
                diversity_penalty=1.0, 
                do_sample=False,
                max_length=128,
            )
            decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            beams_grouped = [
                decoded[j : j + self.num_beams]
                for j in range(0, len(decoded), self.num_beams)
            ]

            flat_srcs = [p[9:] for p in batch_prompts for _ in range(self.num_beams)]
            flat_mts  = [hyp for group in beams_grouped for hyp in group]
            flat_refs = [ref for ref in batch_golds for _ in range(self.num_beams)]
            self.comet.add_batch(predictions=flat_mts, references=flat_refs, sources=flat_srcs)
            scores = self.comet.compute()["scores"]

            grouped_scores = [
                scores[k : k + self.num_beams]
                for k in range(0, len(scores), self.num_beams)
            ]

            for prompt, group, score_list in zip(batch_prompts, beams_grouped, grouped_scores):
                seen = {}
                for hyp, sc in zip(group, score_list):
                    if hyp not in seen:
                        seen[hyp] = sc
                items = sorted(seen.items(), key=lambda x: x[1], reverse=True)
                chosen = items[0][0]
                rejected = items[1][0] if len(items) > 1 else items[0][0]
                if self.ep not in self.save_data:
                    self.save_data[self.ep] = []
                self.save_data[self.ep].append({
                    "prompt": prompt,
                    "chosen": chosen,
                    "rejected": rejected
                })
                if len(self.save_data[self.ep]) == len(self.prompts):
                    self.ep += 1
                yield {
                    "prompt":  prompt,
                    "chosen":  chosen,
                    "rejected": rejected,
                }

                
class DPODataCollator:
    def __init__(self, tokenizer, max_prompt_length=128, max_completion_length=128):
        self.tokenizer = tokenizer
        self.max_prompt_length = max_prompt_length
        self.max_completion_length = max_completion_length

    def __call__(self, batch):
        input_ids = []
        labels_chosen = []
        labels_rejected = []
        pr_am = []
        ch_am = []
        re_am = []

        for example in batch:
            row = CustomDPOTrainer.tokenize_row(
                features=example,
                processing_class=self.tokenizer,
                max_prompt_length=self.max_prompt_length,
                max_completion_length=self.max_completion_length,
                add_special_tokens=True,
            )

            prompt_ids = row["prompt_input_ids"]
            chosen_ids = row["chosen_input_ids"]
            rejected_ids = row["rejected_input_ids"]
            
            prompt_attention_mask = row["prompt_attention_mask"]
            chosen_attention_mask = row["chosen_attention_mask"]
            rejected_attention_mask = row["rejected_attention_mask"]
            
            input_ids.append(prompt_ids)
            labels_chosen.append(chosen_ids)
            labels_rejected.append(rejected_ids)
            pr_am.append(prompt_attention_mask)
            ch_am.append(chosen_attention_mask)
            re_am.append(rejected_attention_mask)
                   
        def pad(batch_ids, pad_val):
            return torch.nn.utils.rnn.pad_sequence(
                [torch.tensor(ids) for ids in batch_ids],
                batch_first=True,
                padding_value=pad_val,
            )
        padded = {
        "prompt_input_ids": pad(input_ids, pad_val=self.tokenizer.pad_token_id),
        "chosen_input_ids": pad(labels_chosen, pad_val=self.tokenizer.pad_token_id),
        "rejected_input_ids": pad(labels_rejected, pad_val=self.tokenizer.pad_token_id),
        "prompt_attention_mask": pad(pr_am, pad_val=0),
        "chosen_attention_mask": pad(ch_am, pad_val=0),
        "rejected_attention_mask": pad(re_am, pad_val=0),
        }
        padded = {k: v.to(device) for k, v in padded.items()}
        return padded                

class CustomDPOTrainer(DPOTrainer):
    def _prepare_dataset(self, dataset, processing_class, args, dataset_name):
        # skipping maps inside
        return dataset

    @staticmethod
    def tokenize_row(features, processing_class, max_prompt_length, max_completion_length, add_special_tokens):
        tokenizer = processing_class
        prompt = tokenizer(features["prompt"],  add_special_tokens=False)
        chosen = tokenizer(features["chosen"],  add_special_tokens=False)
        rejected = tokenizer(features["rejected"], add_special_tokens=False)
        
        p_ids = prompt["input_ids"]
        c_ids = chosen["input_ids"]
        r_ids = rejected["input_ids"]
       
        prompt_attention_mask = prompt["attention_mask"]
        chosen_attention_mask = chosen["attention_mask"]
        rejected_attention_mask = rejected["attention_mask"]
        
        if add_special_tokens:
            if tokenizer.bos_token_id is not None:
                p_ids = [tokenizer.bos_token_id] + p_ids
                prompt_attention_mask = [1] + prompt_attention_mask
            if tokenizer.eos_token_id is not None:
                p_ids = p_ids + [tokenizer.eos_token_id]
                prompt_attention_mask = prompt_attention_mask + [1]

        if tokenizer.eos_token_id is not None:
            c_ids = c_ids + [tokenizer.eos_token_id]
            chosen_attention_mask = chosen_attention_mask + [1]
            r_ids = r_ids + [tokenizer.eos_token_id]
            rejected_attention_mask = rejected_attention_mask + [1]

        if max_prompt_length is not None:
            p_ids = p_ids[-max_prompt_length:]
        if max_completion_length is not None:
            c_ids = c_ids[: max_completion_length]
            r_ids = r_ids[: max_completion_length]

        return {
            "prompt_input_ids": p_ids,
            "prompt_attention_mask": prompt_attention_mask,            
            "chosen_input_ids":  c_ids,
            "chosen_attention_mask": chosen_attention_mask,            
            "rejected_input_ids": r_ids,
            "rejected_attention_mask": rejected_attention_mask,
        }

    def get_train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
        )
    def get_eval_dataloader(self,eval_dataset=None):
        return DataLoader(
            self.eval_dataset,
            batch_size=self.args.per_device_train_batch_size,
            collate_fn=self.data_collator,
            num_workers=self.args.dataloader_num_workers,
        )


train_prompts, train_golds = [a for a in fine_tune['train'][:].values()]
train_prompts = ['grammar: ' + a for a in train_prompts]

# len(train_prompts), len(train_golds)
dev_prompts, dev_golds = [a for a in fine_tune['dev'][:].values()]
dev_prompts = ['grammar: ' + a for a in dev_prompts]

test_prompts, test_golds = [a for a in fine_tune['test'][:].values()]
test_prompts = ['grammar: ' + a for a in test_prompts]    
    
train_ds = OnTheFlyDpoDataset(
    train_prompts,
    train_golds,
    model,
    tokenizer,
    comet_metric,
    num_beams=6,
    batch_size=16,
    num_beam_groups=3
)

dev_ds = OnTheFlyDpoDataset(
    dev_prompts,
    dev_golds,
    model,
    tokenizer,
    comet_metric,
    num_beams=6,
    batch_size=16,
    num_beam_groups=3
)


collator = DPODataCollator(tokenizer)


In [9]:
train_prompts[0], train_golds[0]

('grammar: Первая компания – Даквал-Пули из Худ Ривер , ш .',
 'Первая компания – Даквал-Пули из Худ Ривер , ш .')

In [10]:
len(train_ds)

59947

In [11]:
(len(train_ds) // 16 // 2 + 1)# / 30

1874

In [ ]:
dpo_config = DPOConfig(
    max_steps=1900, 
    beta=0.1,                           
    loss_type='bco_pair', #better sigmoid.
    learning_rate=1e-4,                
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # num_train_epochs=3,
    gradient_accumulation_steps=2,
    eval_strategy="steps",      
    eval_steps=100,                   
    save_strategy="best", 
    metric_for_best_model='loss',
    logging_steps=10,
    fp16=True,
    save_total_limit=1,
    max_length=128,                   
    generate_during_eval=False,
    remove_unused_columns=False,      # Important: keeps prompt/chosen/rejected
    # warmup_ratio=0.02,                 
    lr_scheduler_type="cosine",      
    report_to="all",        
    logging_dir="./logs",
    # padding_value=tokenizer.pad_token_id,  
    output_dir="gec_t5_dpo_A_v3",   
    run_name='gec_t5_dpo_A_bco_pair'
)

In [9]:
trainer = CustomDPOTrainer(
    model=model,
    args=dpo_config,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    data_collator=collator)

In [ ]:
trainer.train()

wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /home/jupyter/work/resources/rugec/notebooks/dpo/wandb/run-20250528_215511-d7d2lotz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run gec_t5_dpo_A_bco_pair
wandb: ⭐️ View project at https://wandb.ai/mika5883/huggingface
wandb: 🚀 View run at https://wandb.ai/mika5883/huggingface/runs/d7d2lotz
  0%|          | 0/1900 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
  0%|          

{'loss': 1.342, 'grad_norm': 1.7331138849258423, 'learning_rate': 9.999446382823013e-05, 'rewards/chosen': -0.0036153055261820555, 'rewards/rejected': -0.11398736387491226, 'rewards/accuracies': 0.6937500238418579, 'rewards/margins': 0.11037204414606094, 'logps/chosen': -1.0418598651885986, 'logps/rejected': -6.1441826820373535, 'logits/chosen': -2.378326654434204, 'logits/rejected': -2.378329038619995, 'epoch': 0.01}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 11/1900 [01:46<4:45:33,  9.07s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 12/1900 [01:55<4:42:25,  8.98s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using:

{'loss': 1.5737, 'grad_norm': 2.1288082599639893, 'learning_rate': 9.998024842193876e-05, 'rewards/chosen': -0.293789803981781, 'rewards/rejected': -0.47767987847328186, 'rewards/accuracies': 0.734375, 'rewards/margins': 0.18389011919498444, 'logps/chosen': -6.779936790466309, 'logps/rejected': -13.15968132019043, 'logits/chosen': -2.8013837337493896, 'logits/rejected': -2.8001739978790283, 'epoch': 0.01}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 21/1900 [03:25<5:07:35,  9.82s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
  1%|          | 22/1900 [03:34<4:58:05,  9.52s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using:

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 17%|█▋        | 321/1900 [1:34:45<5:11:33, 11.84s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 17%|█▋        | 322/1900 [1:34:53<4:45:12, 10.84s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, 

{'loss': 1.666, 'grad_norm': 18.281597137451172, 'learning_rate': 9.291031969832026e-05, 'rewards/chosen': 0.37582749128341675, 'rewards/rejected': 0.08631499111652374, 'rewards/accuracies': 0.703125, 'rewards/margins': 0.28951239585876465, 'logps/chosen': -6.288638591766357, 'logps/rejected': -15.090845108032227, 'logits/chosen': -2.8717195987701416, 'logits/rejected': -2.8729825019836426, 'epoch': 0.18}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 17%|█▋        | 331/1900 [1:36:34<4:30:43, 10.35s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 17%|█▋        | 332/1900 [1:36:43<4:24:42, 10.13s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, 

{'loss': 2.3286, 'grad_norm': 17.521100997924805, 'learning_rate': 9.248010643731935e-05, 'rewards/chosen': 0.26242995262145996, 'rewards/rejected': 0.17432793974876404, 'rewards/accuracies': 0.659375011920929, 'rewards/margins': 0.08810202032327652, 'logps/chosen': -23.099464416503906, 'logps/rejected': -28.480777740478516, 'logits/chosen': -2.843937873840332, 'logits/rejected': -2.843336582183838, 'epoch': 0.18}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 18%|█▊        | 341/1900 [1:38:46<5:57:43, 13.77s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
 18%|█▊        | 342/1900 [1:38:55<5:25:48, 12.55s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, 

In [27]:
trainer.push_to_hub('yet_another_dpo')




spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]
Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]



training_args.bin:   0%|          | 0.00/6.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 90.1k/892M [00:00<17:46, 836kB/s]


spiece.model:   9%|▉         | 90.1k/1.00M [00:00<00:01, 784kB/s]

training_args.bin: 100%|██████████| 6.14k/6.14k [00:00<00:00, 25.0kB/s][A


model.safetensors:   1%|          | 5.53M/892M [00:00<00:52, 17.0MB/s]

model.safetensors:   1%|          | 8.35M/892M [00:00<01:08, 13.0MB/s]

spiece.model: 100%|██████████| 1.00M/1.00M [00:00<00:00, 1.15MB/s]B/s]


model.safetensors:   2%|▏         | 16.0M/892M [00:01<01:09, 12.6MB/s]

model.safetensors:   3%|▎         | 23.9M/892M [00:01<00:38, 22.5MB/s]

model.safetensors:   3%|▎         | 27.3M/892M [00:01<00:57, 15.1MB/s]

model.safetensors:   3%|▎         | 29.9M/892M [00:01<00:52, 16.5MB/s]

model.safetensors:   4%|▎      

CommitInfo(commit_url='https://huggingface.co/mika5883/gec_t5_dpo_A_v2/commit/44914b180551929997b38b30e1ef8568f43eb74c', commit_message='yet_another_dpo', commit_description='', oid='44914b180551929997b38b30e1ef8568f43eb74c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mika5883/gec_t5_dpo_A_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='mika5883/gec_t5_dpo_A_v2'), pr_revision=None, pr_num=None)